# Getting Started with Velour

## Introduction

Velour is a centralized evaluation store which makes it easy to measure, explore, and rank model performance. Velour empowers data scientists and engineers to evaluate the performance of their machine learning pipelines and use those evaluations to make better modeling decisions in the future. For a conceptual introduction to Velour, [check out our project overview](https://striveworks.github.io/velour/).

In this notebook, we'll introduce Velour's high-level abstractions and walk through a computer vision-oriented example of how you can use Velour to evaluate model performance. For task-specific examples, please see our follow-up notebooks below:
- [Tabular classification](https://github.com/Striveworks/velour/blob/main/examples/classification/tabular.ipynb)
- [Object detection](https://github.com/Striveworks/velour/blob/main/examples/object-detection/coco-yolo.ipynb)
- [Semantic segmentation](https://github.com/Striveworks/velour/blob/main/examples/semantic-segmentation/coco-yolo.ipynb)

## High-Level Workflow

Velour is equipped to handle a wide variety of supervised learning tasks thanks to its six core abstractions. We can think of these abstractions as being split into two categories:
- **Dataset**: When describing our actual dataset, we define a `Dataset` containing a list of `GroundTruths` which, in turn, are made up of `Datums` and `Annotations`.
- **Model**: When describing our model outputs, we define a `Model` containing a list of `Predictions` which, in turn, are also made up of `Datums` and `Annotations`. We then link our `Model` to a `Dataset` when finalizing the model.

After we define both our dataset inputs and model outputs, Velour will make it easy to calculate and store our evaluation metrics. Let's start by describing our dataset.

## Defining Our Dataset

To begin, we import all needed packages and connect to our Velour API. For instructions on setting up your API, please see [our docs here](https://striveworks.github.io/velour/getting_started/).

In [1]:
from pathlib import Path

from velour import (
    Client,
    Dataset,
    Model,
    Datum,
    Annotation,
    GroundTruth, 
    Prediction,
    Label,
)
from velour.schemas import (
    BoundingBox, 
    Polygon, 
    BasicPolygon, 
    Point,
)
from velour.enums import TaskType

client = Client("http://0.0.0.0:8000")

Successfully connected to host at http://0.0.0.0:8000/


Next, we define our `Dataset` in Velour.

In [2]:
dataset = Dataset(
    client=client,   
    name="myDataset",
    metadata={        # optional, metadata can take `str`, `int`, `float` value types.
        "some_string": "hello_world",
        "some_number": 1234,
        "a_different_number": 1.234,
    },
    geospatial=None,  # optional, define a GeoJSON
)

To describe the various objects in our `Dataset`, we'll associate a list of `GroundTruths` (made up of `Annotations` and `Datums`) to the `Dataset` we defined above. Note that Velour doesn't actually store any images, and that the `Annotations` we use will vary by our task type (i.e., object detection, semantic segmentation, etc.). For demonstrative purposes, we'll create `GroundTruths` for four different learning tasks in this notebook.

### Creating Object Detection GroundTruths

In [3]:
def create_groundtruth_from_object_detection_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"] 
        }
    )

    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.DETECTION,
            labels=[Label(key="class_label", value=annotation["class_label"])],
            bounding_box=BoundingBox.from_extrema(
                xmin=annotation["bbox"]["xmin"],
                xmax=annotation["bbox"]["xmax"],
                ymin=annotation["bbox"]["ymin"],
                ymax=annotation["bbox"]["ymax"],
            )
        )
        for annotation in element["annotations"]
        if len(annotation) > 0
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

image_object_detections = [
    {"path": "a/b/c/img3.png", "annotations": [{"class_label": "dog", "bbox": {"xmin": 16, "ymin": 130, "xmax": 70, "ymax": 150}}, {"class_label": "person", "bbox": {"xmin": 89, "ymin": 10, "xmax": 97, "ymax": 110}}]},
    {"path": "a/b/c/img4.png", "annotations": [{"class_label": "cat", "bbox": {"xmin": 500, "ymin": 220, "xmax": 530, "ymax": 260}}]},
    {"path": "a/b/c/img5.png", "annotations": []}
]


for element in image_object_detections:
    # create groundtruth
    groundtruth = create_groundtruth_from_object_detection_dict(element)

    # add groundtruth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'img3', 'metadata': {'path': 'a/b/c/img3.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 16.0, 'y': 130.0}, {'x': 70.0, 'y': 130.0}, {'x': 70.0, 'y': 150.0}, {'x': 16.0, 'y': 150.0}]}}, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}, {'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'person', 'score': None}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 89.0, 'y': 10.0}, {'x': 97.0, 'y': 10.0}, {'x': 97.0, 'y': 110.0}, {'x': 89.0, 'y': 110.0}]}}, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img4', 'metadata': {'path': 'a/b/c/img4.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'cat', 's

/Users/nthorlind/git/sw/velour/client/velour/coretypes.py:808: UserWarning: GroundTruth for datum with uid `img5` contains no annotations.
  warnings.warn(


### Creating Image Classification GroundTruths

In [4]:
def create_groundtruth_from_image_classification_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"]
        }
    )

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(key=key, value=value)
                for label in element["annotations"]
                for key, value in label.items()
            ]
        )
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

image_classifications = [
    {"path": "a/b/c/img1.png", "annotations": [{"class_label": "dog"}]},
    {"path": "a/b/c/img2.png", "annotations": [{"class_label": "cat"}]}
]

for element in image_classifications:
    # create groundtruth
    groundtruth = create_groundtruth_from_image_classification_dict(element)

    # add groundtruth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'img1', 'metadata': {'path': 'a/b/c/img1.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img2', 'metadata': {'path': 'a/b/c/img2.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'cat', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}


### Creating Image Segmentation GroundTruths

In [5]:
def create_groundtruth_from_image_segmentation_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"] 
        }
    )

    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.SEGMENTATION,
            labels=[Label(key="class_label", value=annotation["class_label"])],
            polygon=Polygon(
                boundary=BasicPolygon(
                    points=[
                        Point(p["x"], p["y"])
                        for p in annotation["contour"][0]
                    ],
                ),
                holes=[
                    BasicPolygon(
                        points=[
                            Point(p["x"], p["y"])
                            for p in hole
                        ]
                    )
                    for hole in annotation["contour"][1:]
                ]
            )
        )
        for annotation in element["annotations"]
        if len(annotation["contour"]) > 0
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

image_segmentations = [
    {"path": "a/b/c/img6.png", "annotations": [{"class_label": "dog", "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}]]}]},
    {"path": "a/b/c/img7.png", "annotations": [{"class_label": "cat", "contour": [[{"x": 97.2, "y": 40.2}, {"x": 33.33, "y": 44.3}, {"x": 10.9, "y": 18.7}]]}]},
    {"path": "a/b/c/img8.png", "annotations": [{"class_label": "car", "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}], [{"x": 60.0, "y": 15.5}, {"x": 70.9, "y": 50.2}, {"x": 75.9, "y": 28.4}]]}]}
]

for element in image_segmentations:
    # create groundtruth
    groundtruth = create_groundtruth_from_image_segmentation_dict(element)

    # add groundtruth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'img6', 'metadata': {'path': 'a/b/c/img6.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'semantic-segmentation', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': {'boundary': {'points': [{'x': 10.0, 'y': 15.5}, {'x': 20.9, 'y': 50.2}, {'x': 25.9, 'y': 28.4}]}, 'holes': []}, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img7', 'metadata': {'path': 'a/b/c/img7.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'semantic-segmentation', 'labels': [{'key': 'class_label', 'value': 'cat', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': {'boundary': {'points': [{'x': 97.2, 'y': 40.2}, {'x': 33.33, 'y': 44.3}, {'x': 10.9, 'y': 18.7}]}, 'holes': []}, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img8', 'metadata': {'path': 'a/b/c/img8.png'}, 'geospatial':

### Creating Text Classification GroundTruths

In [6]:
def create_groundtruth_from_text_classification_dict(element: dict):
    
    # create Datum using filename, save the full filepath into metadata
    datum = Datum(
        uid=Path(element["path"]).stem,
        metadata={
            "path": element["path"],
            "context": element["annotations"][0]["sentiment"]["context"]
        }
    )

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(
                    key="label", 
                    value=element["annotations"][0]["sentiment"]["label"]
                )
            ]
        )
    ]

    # create and return GroundTruth
    return GroundTruth(
        datum=datum,
        annotations=annotations,
    )

text_classifications = [
    {"path": "a/b/c/text1.txt", "annotations": [{"sentiment": {"context": "Is the content of this product review postive?", "label": "positive"}}]}
]

for element in text_classifications:
    # create groundtruth
    groundtruth = create_groundtruth_from_text_classification_dict(element)

    # add groundtruth to dataset
    dataset.add_groundtruth(groundtruth)
    
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'text1', 'metadata': {'path': 'a/b/c/text1.txt', 'context': 'Is the content of this product review postive?'}, 'geospatial': {}}, 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'label', 'value': 'positive', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}


### Finalizing Our Dataset

Now that we've created all of our `GroundTruth` objects, we finalize our `Dataset` such that it's ready for evaluation. Velour makes finalization a requirement for traceability purposes: we want you to feel confident that a finalized `Dataset` or `Model` won't change over any length of time.



In [7]:
dataset.finalize()

<Response [200]>

## Defining Our Model

Now that we've described our dataset, the next step is to define our model and subsequent predictions. Again, for demonstrative purposes, we'll define predictions for four separate task types in this notebook.

In [8]:
model = Model(
    client=client,
    name="myModel",
    metadata={
        "foo": "bar",
        "some_number": 4321,
    },
    geospatial=None,
)

### Creating Object Detection Predictions

In [9]:
# populate a dictionary mapping Datum UIDs to datums for all of the datums in our dataset
datums_by_uid = {
    datum.uid: datum
    for datum in dataset.get_datums()
}

def create_prediction_from_object_detection_dict(element: dict, datums_by_uid:dict) -> Prediction:
    
    # get datum from dataset using filename
    uid=Path(element["path"]).stem
    datum = datums_by_uid[uid]

    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.DETECTION,
            labels=[
                Label(key="class_label", value=label["class_label"], score=label["score"])
                for label in annotation["labels"]
            ],
            bounding_box=BoundingBox.from_extrema(
                xmin=annotation["bbox"]["xmin"],
                xmax=annotation["bbox"]["xmax"],
                ymin=annotation["bbox"]["ymin"],
                ymax=annotation["bbox"]["ymax"],
            )
        )
        for annotation in element["annotations"]
        if len(annotation) > 0
    ]

    # create and return Prediction
    return Prediction(
        datum=datum,
        annotations=annotations,
    )

object_detections = [
    {"path": "a/b/c/img3.png", "annotations": [
        {"labels": [{"class_label": "dog", "score": 0.8}, {"class_label": "cat", "score": 0.1}, {"class_label": "person", "score": 0.1}], "bbox": {"xmin": 16, "ymin": 130, "xmax": 70, "ymax": 150}}, 
        {"labels": [{"class_label": "dog", "score": 0.05}, {"class_label": "cat", "score": 0.05}, {"class_label": "person", "score": 0.9}], "bbox": {"xmin": 89, "ymin": 10, "xmax": 97, "ymax": 110}}
    ]},
    {"path": "a/b/c/img4.png", "annotations": [
        {"labels": [{"class_label": "dog", "score": 0.8}, {"class_label": "cat", "score": 0.1}, {"class_label": "person", "score": 0.1}], "bbox": {"xmin": 500, "ymin": 220, "xmax": 530, "ymax": 260}}
    ]},
    {"path": "a/b/c/img5.png", "annotations": []}
]

for element in object_detections:
    # create prediction
    prediction = create_prediction_from_object_detection_dict(element, datums_by_uid=datums_by_uid)

    # add prediction to model
    model.add_prediction(dataset, prediction)
    
    print(prediction)

{'datum': {'dataset': 'myDataset', 'uid': 'img3', 'metadata': {'path': 'a/b/c/img3.png'}, 'geospatial': {}}, 'model': 'myModel', 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': 0.8}, {'key': 'class_label', 'value': 'cat', 'score': 0.1}, {'key': 'class_label', 'value': 'person', 'score': 0.1}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 16.0, 'y': 130.0}, {'x': 70.0, 'y': 130.0}, {'x': 70.0, 'y': 150.0}, {'x': 16.0, 'y': 150.0}]}}, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}, {'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': 0.05}, {'key': 'class_label', 'value': 'cat', 'score': 0.05}, {'key': 'class_label', 'value': 'person', 'score': 0.9}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 89.0, 'y': 10.0}, {'x': 97.0, 'y': 10.0}, {'x': 97.0, 'y': 110.0}, {'x': 89.0, 'y': 110.0}]}}, 'polygon': None, 'multipolygon': None, 'rast

/Users/nthorlind/git/sw/velour/client/velour/coretypes.py:1103: UserWarning: Prediction for datum with uid `img5` contains no annotations.
  warnings.warn(


### Creating Image Classification Predictions

In [11]:
def create_prediction_from_image_classification_dict(element: dict, datums_by_uid:dict) -> Prediction:
    
    # get datum from dataset using filename
    uid=Path(element["path"]).stem
    datum = datums_by_uid[uid]

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(key="class_label", value=label["class_label"], score=label["score"])
                for label in element["annotations"]
            ]
        )
    ]

    # create and return Prediction
    return Prediction(
        model=model,
        datum=datum,
        annotations=annotations,
    )

image_classifications = [
    {"path": "a/b/c/img1.png", "annotations": [{"class_label": "dog", "score": 0.9}, {"class_label": "cat", "score": 0.1}]},
    {"path": "a/b/c/img2.png", "annotations": [{"class_label": "dog", "score": 0.1}, {"class_label": "cat", "score": 0.9}]}
]

for element in image_classifications:
    # create prediction
    prediction = create_prediction_from_image_classification_dict(element, datums_by_uid=datums_by_uid)

    # add prediction to dataset
    model.add_prediction(dataset, prediction)
    
    print(prediction)

{'datum': {'dataset': 'myDataset', 'uid': 'img1', 'metadata': {'path': 'a/b/c/img1.png'}, 'geospatial': {}}, 'model': 'myModel', 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': 0.9}, {'key': 'class_label', 'value': 'cat', 'score': 0.1}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img2', 'metadata': {'path': 'a/b/c/img2.png'}, 'geospatial': {}}, 'model': 'myModel', 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': 0.1}, {'key': 'class_label', 'value': 'cat', 'score': 0.9}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}


### Creating Image Segmentation Predictions

In [13]:
def create_prediction_from_image_segmentation_dict(element: dict, datums_by_uid: dict) -> Prediction:
    
    # get datum from dataset using filename
    uid=Path(element["path"]).stem
    datum = datums_by_uid[uid]


    # create Annotations
    annotations = [
        Annotation(
            task_type=TaskType.SEGMENTATION,
            labels=[
                Label(key="class_label", value=annotation["class_label"])
            ],
            polygon=Polygon(
                boundary=BasicPolygon(
                    points=[
                        Point(p["x"], p["y"])
                        for p in annotation["contour"][0]
                    ],
                ),
                holes=[
                    BasicPolygon(
                        points=[
                            Point(p["x"], p["y"])
                            for p in hole
                        ]
                    )
                    for hole in annotation["contour"][1:]
                ]
            )
        )
        for annotation in element["annotations"]
        if len(annotation["contour"]) > 0
    ]

    # create and return Prediction
    return Prediction(
        datum=datum,
        annotations=annotations,
    )

image_segmentations = [
    {
        "path": "a/b/c/img6.png", 
        "annotations": [
            {
                "class_label": "dog",
                "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}]]
            }
        ]
    },
    {
        "path": "a/b/c/img7.png", 
        "annotations": [
            {
                "class_label": "cat",
                "contour": [[{"x": 97.2, "y": 40.2}, {"x": 33.33, "y": 44.3}, {"x": 10.9, "y": 18.7}]]
            }
        ]   
    },
    {
        "path": "a/b/c/img8.png", 
        "annotations": [
            {
                "class_label": "car",
                "contour": [[{"x": 10.0, "y": 15.5}, {"x": 20.9, "y": 50.2}, {"x": 25.9, "y": 28.4}], [{"x": 60.0, "y": 15.5}, {"x": 70.9, "y": 50.2}, {"x": 75.9, "y": 28.4}]]
            }
        ]
    }
]


for element in image_segmentations:
    # create prediction
    prediction = create_prediction_from_image_segmentation_dict(element, datums_by_uid=datums_by_uid)

    # add prediction to model
    model.add_prediction(dataset, prediction)
    
    print(prediction)

{'datum': {'dataset': 'myDataset', 'uid': 'img6', 'metadata': {'path': 'a/b/c/img6.png'}, 'geospatial': {}}, 'model': 'myModel', 'annotations': [{'task_type': 'semantic-segmentation', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': {'boundary': {'points': [{'x': 10.0, 'y': 15.5}, {'x': 20.9, 'y': 50.2}, {'x': 25.9, 'y': 28.4}]}, 'holes': []}, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img7', 'metadata': {'path': 'a/b/c/img7.png'}, 'geospatial': {}}, 'model': 'myModel', 'annotations': [{'task_type': 'semantic-segmentation', 'labels': [{'key': 'class_label', 'value': 'cat', 'score': None}], 'metadata': {}, 'bounding_box': None, 'polygon': {'boundary': {'points': [{'x': 97.2, 'y': 40.2}, {'x': 33.33, 'y': 44.3}, {'x': 10.9, 'y': 18.7}]}, 'holes': []}, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img8', 'metadata': {

### Creating Text Classification Predictions

In [14]:
def create_prediction_from_text_classification_dict(element: dict, datums_by_uid:dict) -> Prediction:
    
    # get datum from dataset using filename
    uid=Path(element["path"]).stem
    datum = datums_by_uid[uid]

    # create Annotation
    annotations = [
        Annotation(
            task_type=TaskType.CLASSIFICATION,
            labels=[
                Label(
                    key="label", 
                    value=label["label"],
                    score=label["score"],
                )
                for label in element["annotations"][0]["sentiment"]["labels"]
            ]
        )
    ]

    # create and return Prediction
    return Prediction(
        datum=datum,
        annotations=annotations,
    )

text_classifications = [
    {
        "path": "a/b/c/text1.txt",
        "annotations": [
            {"sentiment": 
                {
                    "context": "Is the content of this product review postive?", 
                    "labels": [
                        {"label": "positive", "score": 0.8},
                        {"label": "negative", "score": 0.2}
                    ]
                }
            }
        ]
    }
]


for element in text_classifications:
    # create prediction
    prediction = create_prediction_from_text_classification_dict(element, datums_by_uid=datums_by_uid)

    # add prediction to model
    model.add_prediction(dataset, prediction)
    
    print(prediction)

{'datum': {'dataset': 'myDataset', 'uid': 'text1', 'metadata': {'path': 'a/b/c/text1.txt', 'context': 'Is the content of this product review postive?'}, 'geospatial': {}}, 'model': 'myModel', 'annotations': [{'task_type': 'classification', 'labels': [{'key': 'label', 'value': 'positive', 'score': 0.8}, {'key': 'label', 'value': 'negative', 'score': 0.2}], 'metadata': {}, 'bounding_box': None, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}


### Finalizing Our Model

Now that we've created all of our `Prediction` objects, we finalize our `Model` such that it's ready for evaluation. When finalizing our `Model`, we pass in the `Dataset` object that we want to link it to.

In [15]:
model.finalize_inferences(dataset)

## Exploring Our Objects

Now that we've finalized our `Dataset` and `Model`, we can explore all of the objects stored in Velour before running our evaluations.

### Client Exploration

In [16]:
client.get_datasets()

[{'id': 1,
  'name': 'myDataset',
  'metadata': {'some_number': 1234.0,
   'some_string': 'hello_world',
   'a_different_number': 1.234},
  'geospatial': {}}]

In [17]:
client.get_models()

[{'id': 1,
  'name': 'myModel',
  'metadata': {'foo': 'bar', 'some_number': 4321.0},
  'geospatial': {}}]

### Dataset Exploration

In [18]:
for datum in dataset.get_datums():
    print(datum)

{'dataset': 'myDataset', 'uid': 'img3', 'metadata': {'path': 'a/b/c/img3.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img4', 'metadata': {'path': 'a/b/c/img4.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img5', 'metadata': {'path': 'a/b/c/img5.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img1', 'metadata': {'path': 'a/b/c/img1.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img2', 'metadata': {'path': 'a/b/c/img2.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img6', 'metadata': {'path': 'a/b/c/img6.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img7', 'metadata': {'path': 'a/b/c/img7.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'img8', 'metadata': {'path': 'a/b/c/img8.png'}, 'geospatial': {}}
{'dataset': 'myDataset', 'uid': 'text1', 'metadata': {'path': 'a/b/c/text1.txt', 'context': 'Is the content of this product review postive?'}, 'geospatial': {}}


In [19]:
for datum in dataset.get_datums():
    groundtruth = dataset.get_groundtruth(datum)
    print(groundtruth)

{'datum': {'dataset': 'myDataset', 'uid': 'img3', 'metadata': {'path': 'a/b/c/img3.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': None}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 16.0, 'y': 130.0}, {'x': 70.0, 'y': 130.0}, {'x': 70.0, 'y': 150.0}, {'x': 16.0, 'y': 150.0}]}}, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}, {'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'person', 'score': None}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 89.0, 'y': 10.0}, {'x': 97.0, 'y': 10.0}, {'x': 97.0, 'y': 110.0}, {'x': 89.0, 'y': 110.0}]}}, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}]}
{'datum': {'dataset': 'myDataset', 'uid': 'img4', 'metadata': {'path': 'a/b/c/img4.png'}, 'geospatial': {}}, 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'cat', 's

In [20]:
for label in dataset.get_labels():
    print(label)

{'key': 'class_label', 'value': 'person', 'score': None}
{'key': 'class_label', 'value': 'car', 'score': None}
{'key': 'class_label', 'value': 'dog', 'score': None}
{'key': 'label', 'value': 'positive', 'score': None}
{'key': 'class_label', 'value': 'cat', 'score': None}


### Model Exploration

In [21]:
for datum in dataset.get_datums():
    print(model.get_prediction(datum))

{'datum': {'dataset': 'myDataset', 'uid': 'img3', 'metadata': {'path': 'a/b/c/img3.png'}, 'geospatial': {}}, 'model': 'myModel', 'annotations': [{'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': 0.8}, {'key': 'class_label', 'value': 'person', 'score': 0.1}, {'key': 'class_label', 'value': 'cat', 'score': 0.1}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 16.0, 'y': 130.0}, {'x': 70.0, 'y': 130.0}, {'x': 70.0, 'y': 150.0}, {'x': 16.0, 'y': 150.0}]}}, 'polygon': None, 'multipolygon': None, 'raster': None, 'jsonb': None}, {'task_type': 'object-detection', 'labels': [{'key': 'class_label', 'value': 'dog', 'score': 0.05}, {'key': 'class_label', 'value': 'person', 'score': 0.9}, {'key': 'class_label', 'value': 'cat', 'score': 0.05}], 'metadata': {}, 'bounding_box': {'polygon': {'points': [{'x': 89.0, 'y': 10.0}, {'x': 97.0, 'y': 10.0}, {'x': 97.0, 'y': 110.0}, {'x': 89.0, 'y': 110.0}]}}, 'polygon': None, 'multipolygon': None, 'rast

In [22]:
for label in model.get_labels():
    print(label)

{'key': 'class_label', 'value': 'person', 'score': None}
{'key': 'class_label', 'value': 'car', 'score': None}
{'key': 'class_label', 'value': 'dog', 'score': None}
{'key': 'label', 'value': 'positive', 'score': None}
{'key': 'label', 'value': 'negative', 'score': None}
{'key': 'class_label', 'value': 'cat', 'score': None}


## Evaluating Performance

Finally, we'll use our Velour abstractions to evaluate model performance. For more detailed, task-specific examples, see our follow-up notebooks at the links below:
- [Tabular classification](https://github.com/Striveworks/velour/blob/main/examples/classification/tabular.ipynb)
- [Object detection](https://github.com/Striveworks/velour/blob/main/examples/object-detection/coco-yolo.ipynb)
- [Semantic segmentation](https://github.com/Striveworks/velour/blob/main/examples/semantic-segmentation/coco-yolo.ipynb)

### Evaluating Detections

In [23]:
eval_objdet = model.evaluate_detection(dataset)
eval_objdet.wait_for_completion()
eval_objdet.results()

EvaluationResult(dataset='myDataset', model='myModel', settings=EvaluationSettings(parameters=DetectionParameters(iou_thresholds_to_compute=[0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95], iou_thresholds_to_return=[0.5, 0.75]), filters=Filter(dataset_names=None, dataset_metadata=None, dataset_geospatial=None, models_names=None, models_metadata=None, models_geospatial=None, datum_uids=None, datum_metadata=None, datum_geospatial=None, task_types=None, annotation_types=None, annotation_geometric_area=None, annotation_metadata=None, annotation_geospatial=None, prediction_scores=None, labels=None, label_ids=None, label_keys=None)), job_id=1, status='done', metrics=[{'type': 'AP', 'parameters': {'iou': 0.5}, 'value': 1.0, 'label': {'key': 'class_label', 'value': 'person'}}, {'type': 'AP', 'parameters': {'iou': 0.5}, 'value': 1.0, 'label': {'key': 'class_label', 'value': 'dog'}}, {'type': 'AP', 'parameters': {'iou': 0.5}, 'value': 1.0, 'label': {'key': 'class_label', 'value': 'cat'}},

### Evaluating Classifications

Note that running the code below evaluates both our text classifications as well as our image classifications. If we only wanted to evaluate one type of classification task, we could use `evaluation_classification`'s `filters` argument to specify which type of labels to evaluate.

In [24]:
eval_clf = model.evaluate_classification(dataset)
eval_clf.wait_for_completion()
eval_clf.results()

EvaluationResult(dataset='myDataset', model='myModel', settings=EvaluationSettings(parameters=None, filters=None), job_id=2, status='done', metrics=[{'type': 'Accuracy', 'parameters': {'label_key': 'class_label'}, 'value': 1.0}, {'type': 'ROCAUC', 'parameters': {'label_key': 'class_label'}, 'value': 1.0}, {'type': 'Precision', 'value': 1.0, 'label': {'key': 'class_label', 'value': 'dog'}}, {'type': 'Recall', 'value': 1.0, 'label': {'key': 'class_label', 'value': 'dog'}}, {'type': 'F1', 'value': 1.0, 'label': {'key': 'class_label', 'value': 'dog'}}, {'type': 'Precision', 'value': 1.0, 'label': {'key': 'class_label', 'value': 'cat'}}, {'type': 'Recall', 'value': 1.0, 'label': {'key': 'class_label', 'value': 'cat'}}, {'type': 'F1', 'value': 1.0, 'label': {'key': 'class_label', 'value': 'cat'}}, {'type': 'Accuracy', 'parameters': {'label_key': 'label'}, 'value': 1.0}, {'type': 'ROCAUC', 'parameters': {'label_key': 'label'}, 'value': 1.0}, {'type': 'Precision', 'value': 1.0, 'label': {'key'

### Evaluating Segmentations

In [25]:
eval_semseg = model.evaluate_segmentation(dataset)
eval_semseg.wait_for_completion()
eval_semseg.results()

EvaluationResult(dataset='myDataset', model='myModel', settings=EvaluationSettings(parameters=None, filters=Filter(dataset_names=None, dataset_metadata=None, dataset_geospatial=None, models_names=None, models_metadata=None, models_geospatial=None, datum_uids=None, datum_metadata=None, datum_geospatial=None, task_types=None, annotation_types=None, annotation_geometric_area=None, annotation_metadata=None, annotation_geospatial=None, prediction_scores=None, labels=None, label_ids=None, label_keys=None)), job_id=3, status='done', metrics=[{'type': 'mIOU', 'value': -1.0}], confusion_matrices=[])